<a href="https://colab.research.google.com/github/LINGESH-0511/credit_card_default_prediction/blob/main/credit_card_default_predicion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q streamlit pyngrok imbalanced-learn joblib openpyxl xgboost
from pyngrok import ngrok
ngrok.set_auth_token("use your auth token")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 98.4 MB/s eta 0:00:00


In [2]:
import pandas as pd

df = pd.read_csv("default of credit card clients 10000 rows.csv")

df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_", regex=False)
    .str.replace(".", "_", regex=False)
)

print(df.shape)
df.head()


(9999, 25)


,id,limit_bal,sex,education,marriage,age,pay_0,pay_2,pay_3,pay_4,...,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default_payment_next_month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [3]:
TARGET = "default_payment_next_month"

X = df.drop(columns=[TARGET])
y = df[TARGET]


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [5]:
from xgboost import XGBClassifier

model = XGBClassifier(
    n_estimators=300,
    max_depth=3,
    min_child_weight=10,
    gamma=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="logloss",
    random_state=42
)


model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=5,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=10, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, ...)

In [6]:
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score,precision_score

THRESHOLD = 0.1932

y_test_proba = model.predict_proba(X_test)[:, 1]
y_pred = (y_test_proba >= THRESHOLD).astype(int)

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_test_proba)
precision = precision_score(y_test, y_pred)

print("Model Performance (Threshold = 0.1932)")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"ROC AUC  : {roc_auc:.4f}")


Model Performance (Threshold = 0.1932)
Accuracy : 0.7020
Precision: 0.4077
Recall   : 0.7035
ROC AUC  : 0.7707


In [7]:
import joblib

joblib.dump(model, "credit_default_real_model.pkl")
joblib.dump(X_test, "X_test.pkl")
joblib.dump(y_test, "y_test.pkl")

print("✅ Model and test data saved successfully")


✅ Model and test data saved successfully


In [8]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from sklearn.metrics import (
    accuracy_score,
    recall_score,
    roc_auc_score,
    confusion_matrix,
    ConfusionMatrixDisplay
)

THRESHOLD = 0.1932

st.set_page_config(page_title="Credit Default Prediction", layout="wide")
st.title("💳 Credit Card Default Prediction System")

@st.cache_resource
def load_artifacts():
    model = joblib.load("credit_default_real_model.pkl")
    X_test = joblib.load("X_test.pkl")
    y_test = joblib.load("y_test.pkl")
    return model, X_test, y_test

model, X_test, y_test = load_artifacts()
expected_features = list(model.feature_names_in_)



# ================= PAGE CONFIG =================
st.set_page_config(
    page_title="Credit Default Prediction",
    layout="wide"
)

TARGET = "default"

# ================= LOAD MODEL =================
@st.cache_resource
def load_model():
    return joblib.load("credit_default_real_model.pkl")

model = load_model()
expected_features = list(model.feature_names_in_)

# ================= FILE UPLOAD =================
uploaded_file = st.file_uploader(
    "Upload Credit Card Dataset (CSV or Excel)",
    type=["csv", "xlsx"]
)

if uploaded_file:

    # ---------- READ FILE ----------
    if uploaded_file.name.endswith(".csv"):
        df = pd.read_csv(uploaded_file)
    else:
        df = pd.read_excel(uploaded_file)

    # ---------- NORMALIZE COLUMNS ----------
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
    )
    df = df.loc[:, ~df.columns.str.contains("^unnamed")]

    st.subheader("📄 Uploaded Data Preview")
    st.dataframe(df.head())

    # ================= SCHEMA ALIGNMENT =================
    for col in expected_features:
        if col not in df.columns:
            df[col] = 0

    X = df[expected_features]

    # ================= PREDICTION =================
    probabilities = model.predict_proba(X)[:, 1]

    predictions = (probabilities >= THRESHOLD).astype(int)

    df["default_probability"] = probabilities
    df["predicted_default"] = predictions


    st.markdown("### 📈 Prediction Confidence & Risk Summary")

    avg_prob = probabilities.mean()
    high_risk = (probabilities >= 0.7).sum()
    medium_risk = ((probabilities >= 0.3) & (probabilities < 0.7)).sum()
    low_risk = (probabilities < 0.3).sum()

    st.write(f"**Average Default Probability:** {avg_prob:.4f}")
    st.write(f"**High Risk Customers (≥ 0.7):** {high_risk}")
    st.write(f"**Medium Risk Customers (0.3 – 0.7):** {medium_risk}")
    st.write(f"**Low Risk Customers (< 0.3):** {low_risk}")

    # Risk distribution bar chart
    fig, ax = plt.subplots(figsize=(5, 4))
    ax.bar(
        ["Low Risk", "Medium Risk", "High Risk"],
        [low_risk, medium_risk, high_risk]
    )
    ax.set_ylabel("Number of Customers")
    ax.set_title("Risk Distribution (Model Confidence)")
    st.pyplot(fig)

    # ================= SEARCH BY CUSTOMER ID =================
    st.subheader("🔍 Search by Customer ID")
    search_id = st.text_input("Enter Customer ID")

    if search_id and "id" in df.columns:
        result = df[df["id"].astype(str) == search_id]
        if not result.empty:
            prob = result["default_probability"].values[0]
            if prob >= 0.7:
                risk = "High Risk"
            elif prob >= 0.3:
                risk = "Medium Risk"
            else:
                risk = "Low Risk"

            result = result.copy()
            result["Risk_Bucket"] = risk

            st.dataframe(result)
            st.success(f"Customer falls into: {risk}")
        else:
            st.warning("Customer not found")

    # ================= DEFAULTERS =================
    defaulters = df[df["predicted_default"] == 1]

    st.subheader("🚨 Predicted Defaulters")
    st.dataframe(defaulters)
    st.success(f"Total Defaulters: {len(defaulters)}")


    # ================= DOWNLOAD =================
    defaulters.to_excel("defaulters_output.xlsx", index=False)
    with open("defaulters_output.xlsx", "rb") as f:
        st.download_button(
            "📥 Download Defaulters Excel",
            f,
            file_name="defaulters_output.xlsx"
        )


    # ================= PIE CHART =================
    st.subheader("📊 Default vs Non-Default Distribution")
    counts = df["predicted_default"].value_counts()

    fig2, ax2 = plt.subplots(figsize=(5, 5))
    ax2.pie(
        [counts.get(0, 0), counts.get(1, 0)],
        labels=["Non-Default", "Default"],
        autopct="%1.1f%%",
        startangle=90
    )
    st.pyplot(fig2)

else:
    st.info("📂 Please upload a file to see predictions and analytics.")


# ================= CONFUSION MATRIX =================
y_test_proba = model.predict_proba(X_test)[:, 1]
y_test_pred = (y_test_proba >= THRESHOLD).astype(int)

cm = confusion_matrix(y_test, y_test_pred)

st.subheader("🧮 Confusion Matrix (Threshold = 0.1932)")
fig, ax = plt.subplots(figsize=(5, 4))
disp = ConfusionMatrixDisplay(cm, display_labels=["Non-Default", "Default"])
disp.plot(ax=ax, cmap="Blues", values_format="d")
st.pyplot(fig)



Writing app.py


In [9]:
from pyngrok import ngrok

ngrok.kill()

public_url = ngrok.connect(8501, proto="http")
print("🚀 Streamlit URL:", public_url)

!streamlit run app.py --server.port 8501 --server.enableCORS false


🚀 Streamlit URL: NgrokTunnel: "https://oligopolistic-nubia-overjudiciously.ngrok-free.dev" -> "http://localhost:8501"
2025-12-29 15:07:25.153 
'server.enableXsrfProtection=true'.
As a result, 'server.enableCORS' is being overridden to 'true'.

More information:
In order to protect against CSRF attacks, we send a cookie with each request.
To do so, we must specify allowable origins, which places a restriction on
cross-origin resource sharing.

If cross origin resource sharing is required, please disable server.enableXsrfProtection.
            





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.12.20.184:8501

  Stopping...
